# PRIMO 2022 ML Homework 4<br>
### Estimating the issue priority in the bug tracker

Задача: предсказать, имеет ли проблема высокий приоритет, учитывая ее описание и метаданные.<br>
Данные: взяты с сайта youtrack.jetbrains.com.<br>
Оценка качества: F1 мера

Метрика оценки качества для соревнования: F1 мера для класса 1.<br>
<br>
**Формат файла с предсказаниями<br>**
Для каждой задачи в наборе test итоговый файл с предсказаниями должен содержать два значения: id (id задачи, взятый из столбца `id` в наборе данных test) и предсказание приоритета задачи (1, если приоритет высокий, или 0, если приоритет низкий).<br>
<br>
Файл должен иметь заголовок и следующий формат:<br>
<br>
id,is_high_priority<br>
0,1<br>
1,1<br>
2,1<br>

In [1]:
import pandas as pd
import numpy as np

import warnings
import json
import operator

from tqdm import tqdm

from sklearn.model_selection import train_test_split


from sklearn.model_selection import ParameterGrid
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE, SelectKBest, f_regression
from sklearn.naive_bayes import MultinomialNB


from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA, NMF
from sklearn.feature_selection import SelectKBest, SelectPercentile, chi2
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import classification_report
#np.set_printoptions(precision=3)
#pd.set_option('precision', 3)



In [2]:
test_data = pd.read_csv('test.csv', index_col='id')


In [3]:
test_data_ind = pd.read_csv('test.csv')

In [4]:
train_data = pd.read_csv('train.csv', index_col='id')

In [5]:
train_data.summary.fillna('', inplace=True)
train_data.description.fillna('', inplace=True)


In [6]:
test_data

,summary,description,reporter,created,customFields,links
id,,,,,,
25-2512303,WSL support for JVM development under Windows,As a Kotlin/Scala/Java developer on Windows I ...,"{""login"": ""joost.3"", ""$type"": ""User""}",1569949612582,"[{""value"": {""name"": ""Feature"", ""$type"": ""EnumB...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
25-2513780,Mozilla's Fluent (*.ftl) syntax highlighting,What steps will reproduce the problem?\n\n1. O...,"{""login"": ""tobiaskybernium"", ""$type"": ""User""}",1570088574970,"[{""value"": {""name"": ""Feature"", ""$type"": ""EnumB...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
25-2515293,Quick definition (Ctrl+Shift+I) called on Lomb...,What steps will reproduce the issue?\n1. Creat...,"{""login"": ""Sergei_Tsypanov"", ""$type"": ""User""}",1570190482389,"[{""value"": {""name"": ""Bug"", ""$type"": ""EnumBundl...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
25-2514349,Checkboxes in search dialog not aligned to pix...,Ugly checkbox boundaries under windows 10 (per...,"{""login"": ""gamalik"", ""$type"": ""User""}",1570110634652,"[{""value"": {""name"": ""Cosmetics"", ""$type"": ""Enu...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
25-2514337,HTTP client: Response Handler is not called wh...,**Problem**\n\nI can't use [response handler s...,"{""login"": ""iwansoft"", ""$type"": ""User""}",1570110085315,"[{""value"": {""name"": ""Usability Problem"", ""$typ...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
...,...,...,...,...,...,...
25-3224524,Incorrect Error Message Text Decoding of Java ...,Intellij uses incorrect text encoding for disp...,"{""login"": ""wh1099"", ""$type"": ""User""}",1616373544355,"[{""value"": {""name"": ""Configuration Problem"", ""...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
25-3224569,Code Style: Spacing for code braces not work,**What steps will reproduce the issue?**\n\n1....,"{""login"": ""blackj0221"", ""$type"": ""User""}",1616385044997,"[{""value"": {""name"": ""Bug"", ""$type"": ""EnumBundl...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
25-3224564,Diglog is hided by the taskbar when taskbar is...,Intellij IDEA 2020.3.3 \nwindows 10\n\n**What ...,"{""login"": ""myfirenze"", ""$type"": ""User""}",1616382740184,"[{""value"": {""name"": ""Bug"", ""$type"": ""EnumBundl...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."


mylist = []

for chunk in  pd.read_csv('train_2011_2012_2013.csv', sep=';', chunksize=20000):
    mylist.append(chunk)

big_data = pd.concat(mylist, axis= 0)
del mylist

In [7]:
train_data1 = train_data.sample(frac = 1)

In [8]:
chunks= np.array_split(train_data1, 5)

In [9]:
train_data1 = chunks[0]

In [10]:
#chunks= np.array_split(train_data, 2)

In [11]:
#train_data1 = chunks[0]

In [12]:
test_data

,summary,description,reporter,created,customFields,links
id,,,,,,
25-2512303,WSL support for JVM development under Windows,As a Kotlin/Scala/Java developer on Windows I ...,"{""login"": ""joost.3"", ""$type"": ""User""}",1569949612582,"[{""value"": {""name"": ""Feature"", ""$type"": ""EnumB...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
25-2513780,Mozilla's Fluent (*.ftl) syntax highlighting,What steps will reproduce the problem?\n\n1. O...,"{""login"": ""tobiaskybernium"", ""$type"": ""User""}",1570088574970,"[{""value"": {""name"": ""Feature"", ""$type"": ""EnumB...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
25-2515293,Quick definition (Ctrl+Shift+I) called on Lomb...,What steps will reproduce the issue?\n1. Creat...,"{""login"": ""Sergei_Tsypanov"", ""$type"": ""User""}",1570190482389,"[{""value"": {""name"": ""Bug"", ""$type"": ""EnumBundl...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
25-2514349,Checkboxes in search dialog not aligned to pix...,Ugly checkbox boundaries under windows 10 (per...,"{""login"": ""gamalik"", ""$type"": ""User""}",1570110634652,"[{""value"": {""name"": ""Cosmetics"", ""$type"": ""Enu...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
25-2514337,HTTP client: Response Handler is not called wh...,**Problem**\n\nI can't use [response handler s...,"{""login"": ""iwansoft"", ""$type"": ""User""}",1570110085315,"[{""value"": {""name"": ""Usability Problem"", ""$typ...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
...,...,...,...,...,...,...
25-3224524,Incorrect Error Message Text Decoding of Java ...,Intellij uses incorrect text encoding for disp...,"{""login"": ""wh1099"", ""$type"": ""User""}",1616373544355,"[{""value"": {""name"": ""Configuration Problem"", ""...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
25-3224569,Code Style: Spacing for code braces not work,**What steps will reproduce the issue?**\n\n1....,"{""login"": ""blackj0221"", ""$type"": ""User""}",1616385044997,"[{""value"": {""name"": ""Bug"", ""$type"": ""EnumBundl...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
25-3224564,Diglog is hided by the taskbar when taskbar is...,Intellij IDEA 2020.3.3 \nwindows 10\n\n**What ...,"{""login"": ""myfirenze"", ""$type"": ""User""}",1616382740184,"[{""value"": {""name"": ""Bug"", ""$type"": ""EnumBundl...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."


In [13]:
#train_data1 = train_data.sample(frac = 1)
#chunks = np.array_split(train_data1, 10)
#train_data1 = chunks[0]

In [14]:
train_data1.summary.fillna('', inplace=True)
train_data1.description.fillna('', inplace=True)


In [15]:
test_data.summary.fillna('', inplace=True)
test_data.description.fillna('', inplace=True)

In [16]:
json_fields = pd.json_normalize(train_data1.customFields.map(json.loads).map(lambda x: {field['name']: field['value'] for field in x}))
json_fields.set_index(train_data1.index, inplace=True)
json_fields.fillna('',inplace=True)



In [17]:
json_fields_test = pd.json_normalize(test_data.customFields.map(json.loads).map(lambda x: {field['name']: field['value'] for field in x}))
json_fields_test.set_index(test_data.index, inplace=True)
json_fields.fillna('',inplace=True)



In [18]:
json_fields['Subsystem.name'].value_counts()

                        4586
User Interface          1399
Editor. Editing Text     828
Build. Maven             706
Lang. Flash and Flex     704
                        ... 
Core. Image Viewer         1
Clouds                     1
Editor. Large Files        1
Core                       1
Editor. Copyright          1
Name: Subsystem.name, Length: 168, dtype: int64

In [19]:
json_columns = ['Type.name','State.name','Subsystem.name','Assignee.login']
train_data1 = train_data1.join(json_fields[json_columns], how='outer', lsuffix='_left')


train_data1['reporter_name'] = train_data1.reporter.map(json.loads).map(operator.itemgetter('login'))
train_data1['Year'] = train_data1.created.apply(lambda c: pd.to_datetime(c, unit='ms').year)
train_data1['Month'] = train_data1.created.apply(lambda c: pd.to_datetime(c, unit='ms').month)
train_data1['Day'] = train_data1.created.apply(lambda c: pd.to_datetime(c, unit='ms').day)
train_data1['Hour'] = train_data1.created.apply(lambda c: pd.to_datetime(c, unit='ms').hour)
prep_train_data = train_data1.drop(labels=['reporter', 'created', 'customFields', 'links'], axis=1)

In [20]:
json_columns = ['Type.name','State.name','Subsystem.name','Assignee.login']
train_data_test = train_data.join(json_fields[json_columns], how='outer', lsuffix='_left')


train_data_test['Year'] = train_data_test.created.apply(lambda c: pd.to_datetime(c, unit='ms').year)
train_data_test['Month'] = train_data_test.created.apply(lambda c: pd.to_datetime(c, unit='ms').month)
train_data_test['Day'] = train_data_test.created.apply(lambda c: pd.to_datetime(c, unit='ms').day)
train_data_test['Hour'] = train_data_test.created.apply(lambda c: pd.to_datetime(c, unit='ms').hour)
prep_train_data_test = train_data_test.drop(labels=['reporter', 'created', 'customFields', 'links'], axis=1)

In [21]:
json_columns_test = ['Type.name','State.name','Subsystem.name','Assignee.login']
test_data= test_data.join(json_fields_test[json_columns_test], how='outer', lsuffix='_left')


test_data['reporter_name'] = test_data.reporter.map(json.loads).map(operator.itemgetter('login'))
test_data['Year'] = test_data.created.apply(lambda c: pd.to_datetime(c, unit='ms').year)
test_data['Month'] = test_data.created.apply(lambda c: pd.to_datetime(c, unit='ms').month)
test_data['Day'] = test_data.created.apply(lambda c: pd.to_datetime(c, unit='ms').day)
test_data['Hour'] = test_data.created.apply(lambda c: pd.to_datetime(c, unit='ms').hour)
prep_test_data = test_data.drop(labels=['reporter', 'created', 'customFields', 'links'], axis=1)

In [22]:
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import re
from sklearn.feature_selection import SelectFromModel

In [23]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\WildGoose\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\WildGoose\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\WildGoose\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [24]:
class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
       return [self.wnl.lemmatize(t) for t in word_tokenize(doc) if re.fullmatch(r'[a-zA-Z0-9]\w\S{4,}', t) and t not in STOP_WORDS]

In [25]:
X_train_text = prep_train_data.summary.astype(str) +prep_train_data.description.astype(str)


In [26]:
X_train_text

id
25-2031497    Spring SpEL cannot resolve variable issueWhat ...
25-182031     Security alert can't be closedI've changed my ...
25-510848     Run configuration window mangled with YourKit ...
25-670478     Git merge does not show files that would be ov...
25-1450768    Autocomplete and code navigation during reinde...
                                    ...                        
25-1520924    autoscroll project tool window (auto horizonta...
25-2020190    Window focus swapping on Search Everywhere wit...
25-234331     Change default indentation for groovy and GSP ...
25-2014553    Cannot go from Find to Replace if shortcut was...
25-983441     Property duplicates in pom.xmlRecently I faced...
Length: 21073, dtype: object

In [27]:
X_test_text = prep_test_data.summary.astype(str) + " " + prep_test_data.description.astype(str)

In [28]:
X_test_text

id
25-2512303    WSL support for JVM development under Windows ...
25-2513780    Mozilla's Fluent (*.ftl) syntax highlighting W...
25-2515293    Quick definition (Ctrl+Shift+I) called on Lomb...
25-2514349    Checkboxes in search dialog not aligned to pix...
25-2514337    HTTP client: Response Handler is not called wh...
                                    ...                        
25-3224524    Incorrect Error Message Text Decoding of Java ...
25-3224569    Code Style: Spacing for code braces not work *...
25-3224564    Diglog is hided by the taskbar when taskbar is...
25-3224549    Debugger shall use "MethodParameters" attribut...
25-3224582    InteliJIdea Ultimate Keyboard shortcuts not wo...
Length: 18529, dtype: object

In [29]:
tfidf_vec_summary = TfidfVectorizer(min_df=.005, max_df=0.9,
                                    tokenizer=LemmaTokenizer(),
                             )
tfidf_vec_summary.fit(X_train_text)
train_text_prep = pd.DataFrame(tfidf_vec_summary.transform(X_train_text).todense(), index=X_train_text.index,
                               columns=tfidf_vec_summary.get_feature_names_out())
train_text_prep



c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,11.0.3,1440x900,1920x1080,2016.3,2017.1,2017.2,2017.3,2018.1,2018.2,2018.3,...,webstorm,window,workaround,worked,worker.run,worker.runtask,working,writing,written,x86_64
id,,,,,,,,,,,,,,,,,,,,,
25-2031497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25-182031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25-510848,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.288635,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25-670478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25-1450768,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-1520924,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.088537,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25-2020190,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.275984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25-234331,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
train_text_prep['1920x1080'].unique()

array([0.        , 0.29088018, 0.18257951, 0.12650314, 0.34647335,
       0.53144828, 0.22883062, 0.18644094, 0.27140301, 0.63979167,
       0.26842744, 0.25781584, 0.65203941, 0.22552182, 0.42218202,
       0.20488335, 0.11818196, 0.24201872, 0.30579287, 0.20046118,
       0.27562656, 0.29536706, 0.10526372, 0.64342532, 0.44535857,
       0.27445693, 0.36312576, 0.38730134, 0.19810252, 0.79698677,
       0.2420734 , 0.25766903, 0.20437488, 0.30387017, 0.14157491,
       0.3415628 , 0.46361458, 0.19742144, 0.13825162, 0.62059207,
       0.89297827, 0.69432613, 0.44220152, 0.28682962, 0.50363696,
       0.68325558, 0.32679844, 0.17685311, 0.44972647, 0.6479798 ,
       0.74877241, 0.30205455, 0.48686509, 0.50622738, 0.60541335,
       0.11060179, 0.16482285, 0.06918033, 0.39080174, 0.28991315,
       0.21797097, 0.14171773, 0.12174404, 0.2744089 , 0.48963912,
       0.545458  , 0.57537562, 0.25813051, 0.21730047, 0.5570838 ,
       0.5520624 , 0.65393207, 0.28075913, 0.23317732, 0.41503

In [31]:
test_text_prep = pd.DataFrame(tfidf_vec_summary.transform(X_test_text).todense(), index=X_test_text.index,
                               columns=tfidf_vec_summary.get_feature_names_out())

In [32]:
test_text_prep

,11.0.3,1440x900,1920x1080,2016.3,2017.1,2017.2,2017.3,2018.1,2018.2,2018.3,...,webstorm,window,workaround,worked,worker.run,worker.runtask,working,writing,written,x86_64
id,,,,,,,,,,,,,,,,,,,,,
25-2512303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.597714,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
25-2513780,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
25-2515293,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
25-2514349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.513817,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
25-2514337,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-3224524,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.275785,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
25-3224569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
25-3224564,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.158420,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000


In [33]:
prep_train_data

,summary,description,is_high_priority,Type.name,State.name,Subsystem.name,Assignee.login,reporter_name,Year,Month,Day,Hour
id,,,,,,,,,,,,
25-2031497,Spring SpEL cannot resolve variable issue,What steps will reproduce the issue?\n1.Have a...,False,Bug,Answered,Frameworks. Spring,sergeyv,jefflage,2018,6,26,18
25-182031,Security alert can't be closed,I've changed my local SVN repository certifica...,False,Bug,Obsolete,Version Control,Kirill.Likhodedov,szymonb,2010,5,31,8
25-510848,Run configuration window mangled with YourKit ...,I'm not sure whether this is a YourKit plugin ...,False,Bug,Can't Reproduce,User Interface,Vassiliy.Kudryashov,mers,2013,2,15,10
25-670478,Git merge does not show files that would be ov...,I have IDEA 13.0.2\nRunning git merge popups a...,False,Bug,Duplicate,Version Control. Git,Kirill.Likhodedov,Jacek.Kolodziejczyk,2014,2,28,10
25-1450768,Autocomplete and code navigation during reinde...,Reindexing of big projects takes a lot of time...,False,Usability Problem,Duplicate,Editor. Code Completion,peter,Volodymyr.Kornilov,2017,2,20,13
...,...,...,...,...,...,...,...,...,...,...,...,...
25-1520924,autoscroll project tool window (auto horizonta...,project tool window PTW has functionality to s...,False,Feature,Shelved,User Interface. Project View,Sergey_Malenkov,darth_pixel,2017,4,24,6
25-2020190,Window focus swapping on Search Everywhere wit...,The issue occurs intermittently (me and my tea...,False,Bug,Duplicate,User Interface. Focus,fokin,mszmatuszewski,2018,6,13,20
25-234331,Change default indentation for groovy and GSP ...,The default indentation for groovy and GSP fil...,False,Bug,Fixed,,peter,James.Cook,2011,1,7,21


tfidf_vec_summary = TfidfVectorizer(min_df=.005, max_df=0.75,
                                    tokenizer=LemmaTokenizer(),
                             )

tfidf_vec_summary.fit_transform(X_train_text)


tfidf_vec_summary.fit(X_test_text)
test_text_prep = pd.DataFrame(tfidf_vec_summary.transform(X_test_text).todense(), index=X_test_text.index,
                               columns=tfidf_vec_summary.get_feature_names_out())
test_text_prep


In [34]:
Y = train_data1['is_high_priority']

In [35]:
Y

id
25-2031497    False
25-182031     False
25-510848     False
25-670478     False
25-1450768    False
              ...  
25-1520924    False
25-2020190    False
25-234331     False
25-2014553    False
25-983441     False
Name: is_high_priority, Length: 21073, dtype: bool

In [36]:
# X_train, X_test, y_train, y_test = train_test_split(train_text_prep, Y,
#                                                     stratify=Y,
#                                                     random_state=3,
#                                                     test_size=0.3,
#                                                     shuffle=True)

In [37]:
prep_train_data.drop(['description','summary','is_high_priority','reporter_name'], axis=1, inplace =True)

In [38]:


preprocessor = DataFrameMapper([
                                (['Type.name'], OneHotEncoder(sparse=False, handle_unknown='ignore', dtype=np.single)),
                                (['State.name'], OneHotEncoder(sparse=False, handle_unknown='ignore',dtype=np.single)),
                                (['Subsystem.name'], OneHotEncoder(sparse=False, handle_unknown='ignore',dtype=np.single)),
                                (['Assignee.login'],OneHotEncoder(sparse=False, handle_unknown='ignore',dtype=np.single)),
                                ('Year', None),
                                ('Month', None),
                                ('Day', None),
                                ('Hour', None),
                                ],
                               input_df=True,
                               df_out=True)



In [39]:
prep_train_data = preprocessor.fit_transform(prep_train_data)

c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\de

In [40]:
prep_test_data = preprocessor.transform(prep_test_data)

c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\de

In [41]:

prep_test_data

,Type.name_x0_Auto-reported Exception,Type.name_x0_Bug,Type.name_x0_Configuration Problem,Type.name_x0_Cosmetics,Type.name_x0_Exception,Type.name_x0_Feature,Type.name_x0_Meta Issue,Type.name_x0_Performance Problem,Type.name_x0_Security Problem,Type.name_x0_Support Request,...,Assignee.login_x0_yannc76,Assignee.login_x0_yole,Assignee.login_x0_yopox,Assignee.login_x0_yuriy.artamonov,Assignee.login_x0_zollycute,Assignee.login_x0_zolotov,Year,Month,Day,Hour
id,,,,,,,,,,,,,,,,,,,,,
25-2512303,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,2019,10,1,17
25-2513780,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,2019,10,3,7
25-2515293,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2019,10,4,12
25-2514349,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2019,10,3,13
25-2514337,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2019,10,3,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-3224524,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2021,3,22,0
25-3224569,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2021,3,22,3
25-3224564,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2021,3,22,3


In [42]:
prep_train_data

,Type.name_x0_Auto-reported Exception,Type.name_x0_Bug,Type.name_x0_Configuration Problem,Type.name_x0_Cosmetics,Type.name_x0_Exception,Type.name_x0_Feature,Type.name_x0_Meta Issue,Type.name_x0_Performance Problem,Type.name_x0_Security Problem,Type.name_x0_Support Request,...,Assignee.login_x0_yannc76,Assignee.login_x0_yole,Assignee.login_x0_yopox,Assignee.login_x0_yuriy.artamonov,Assignee.login_x0_zollycute,Assignee.login_x0_zolotov,Year,Month,Day,Hour
id,,,,,,,,,,,,,,,,,,,,,
25-2031497,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2018,6,26,18
25-182031,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2010,5,31,8
25-510848,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2013,2,15,10
25-670478,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2014,2,28,10
25-1450768,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2017,2,20,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-1520924,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2017,4,24,6
25-2020190,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2018,6,13,20
25-234331,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2011,1,7,21


In [43]:
train_text_prep

,11.0.3,1440x900,1920x1080,2016.3,2017.1,2017.2,2017.3,2018.1,2018.2,2018.3,...,webstorm,window,workaround,worked,worker.run,worker.runtask,working,writing,written,x86_64
id,,,,,,,,,,,,,,,,,,,,,
25-2031497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25-182031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25-510848,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.288635,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25-670478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25-1450768,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-1520924,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.088537,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25-2020190,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.275984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25-234331,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
#prep_train_data = prep_train_data.drop(['summary','description'], axis=1)

In [45]:
prep_train_data = prep_train_data.join(train_text_prep)

In [46]:
prep_test_data = prep_test_data.join(test_text_prep)

prep_test_data = prep_test_data.drop(['summary','description'], axis=1)
prep_test_data = prep_test_data.join(test_text_prep)

In [47]:

prep_train_data

,Type.name_x0_Auto-reported Exception,Type.name_x0_Bug,Type.name_x0_Configuration Problem,Type.name_x0_Cosmetics,Type.name_x0_Exception,Type.name_x0_Feature,Type.name_x0_Meta Issue,Type.name_x0_Performance Problem,Type.name_x0_Security Problem,Type.name_x0_Support Request,...,webstorm,window,workaround,worked,worker.run,worker.runtask,working,writing,written,x86_64
id,,,,,,,,,,,,,,,,,,,,,
25-2031497,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25-182031,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25-510848,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.288635,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25-670478,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25-1450768,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-1520924,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.088537,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25-2020190,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.275984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25-234331,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
#pd.options.display.max_rows = 200

Y

id
25-2031497    False
25-182031     False
25-510848     False
25-670478     False
25-1450768    False
              ...  
25-1520924    False
25-2020190    False
25-234331     False
25-2014553    False
25-983441     False
Name: is_high_priority, Length: 21073, dtype: bool

tfidf_vec_desc = TfidfVectorizer(min_df=.01, max_df=0.5,
                                   # token_pattern=r'[A-Za-z]{2,}',
                                    tokenizer=LemmaTokenizer(),
                                    stop_words=STOP_WORDS)
tfidf_vec_desc.fit(X_train_description)
X_train_description_sample = pd.DataFrame(tfidf_vec_desc.transform(X_train_description).todense(),
                                          columns=tfidf_vec_desc.get_feature_names_out())
X_train_description_sample

\# todo сократить размерность. Лемматизация, регулярные выражения поумнее, пороги для векторизации, объединить слова описания и заголовка?
отбор признаков по модели

In [49]:
X = prep_train_data


In [50]:
#dataFrameMapper_data = preprocessor.fit_transform(X)


In [51]:
#dataFrameMapper_data.to_csv('dataMapperData.csv')

In [52]:
#Y = Y.astype(np.int8)


In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                    stratify=Y,
                                                    random_state=3,
                                                    test_size=0.25,
                                                    shuffle=True)

In [54]:
prep_test_data

,Type.name_x0_Auto-reported Exception,Type.name_x0_Bug,Type.name_x0_Configuration Problem,Type.name_x0_Cosmetics,Type.name_x0_Exception,Type.name_x0_Feature,Type.name_x0_Meta Issue,Type.name_x0_Performance Problem,Type.name_x0_Security Problem,Type.name_x0_Support Request,...,webstorm,window,workaround,worked,worker.run,worker.runtask,working,writing,written,x86_64
id,,,,,,,,,,,,,,,,,,,,,
25-2512303,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.597714,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
25-2513780,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
25-2515293,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
25-2514349,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.513817,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
25-2514337,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-3224524,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.275785,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
25-3224569,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
25-3224564,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.158420,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000


In [55]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sfm = SelectFromModel(LinearSVC(max_iter=2000, dual=False))
check_data = sfm.fit_transform(X_train, y_train)

In [ ]:
check_data

In [ ]:
LR_test = Pipeline([
        ('feature_selection', SelectFromModel(LinearSVC())),
        ('classification', LogisticRegression())
    ])
LR_test.fit(X,Y)

In [ ]:
prediction_test = LR_test.predict_proba(prep_test_data)[:,1]

In [58]:
from sklearn.ensemble import RandomForestRegressor


In [189]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LassoCV


In [210]:
LR= Pipeline([
       ('feature_selection', SelectFromModel(LinearSVC())),
        ('classification', LogisticRegression(solver='lbfgs', multi_class='ovr',max_iter=1000,C=0.1,penalty="l2"))
    ])
LR.fit(X_train,y_train)

c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('feature_selection', SelectFromModel(estimator=LinearSVC())),
                ('classification',
                 LogisticRegression(C=0.1, max_iter=1000, multi_class='ovr'))])

In [233]:
from sklearn.neural_network import MLPClassifier

NN = MLPClassifier(solver='lbfgs', alpha=1e-3, hidden_layer_sizes=(100, 50), random_state=1)
NN.fit(X_train, y_train)


c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(alpha=0.001, hidden_layer_sizes=(100, 50), random_state=1,
              solver='lbfgs')

In [234]:
proba_train = NN.predict_proba(X_test)[:,1]

In [230]:
proba_train

array([0.04323217, 0.04323217, 0.04323217, ..., 0.04323217, 0.04323217,
       0.04323217])

In [235]:
train_f1 = f1_score(y_true=y_test, y_pred=proba_train > 0.00)

In [236]:
train_f1

0.08295433873021647

In [496]:
prediction_test = prediction_test> 0.115

In [497]:
prediction_test

array([False, False, False, ..., False, False, False])

In [200]:
test_data_indexes = test_data_ind.id

In [201]:
test_data_indexes

0        25-2512303
1        25-2513780
2        25-2515293
3        25-2514349
4        25-2514337
            ...    
18524    25-3224524
18525    25-3224569
18526    25-3224564
18527    25-3224549
18528    25-3224582
Name: id, Length: 18529, dtype: object

In [501]:
dfInd = pd.DataFrame(test_data_indexes)

In [502]:
dfPred = pd.DataFrame(prediction_test, columns=['is_high_priority'])

In [503]:
test =  pd.concat([dfInd,dfPred], axis=1)

In [504]:
test

,id,is_high_priority
0,25-2512303,False
1,25-2513780,False
2,25-2515293,False
3,25-2514349,False
4,25-2514337,False
...,...,...
18524,25-3224524,False
18525,25-3224569,False
18526,25-3224564,False
18527,25-3224549,False


In [505]:
test.to_csv('predictions2.csv',index=False)

In [498]:
unique, counts = np.unique(prediction_test, return_counts=True)

In [499]:
print(np.asarray((unique, counts)))

[[    0     1]
 [17672   857]]


In [500]:
857/17672

0.04849479402444545

In [130]:
prediction_train = LR.predict_proba(X_test)[:,1]

In [134]:
test_f1 = f1_score(y_true=y_test, y_pred=prediction_train > 0.07)

In [195]:
unique_train, counts_train = np.unique(y_test, return_counts=True)


In [196]:
print(np.asarray((unique_train, counts_train)))

[[   0    1]
 [5051  218]]


In [197]:
218/5051

0.04315977034250643

In [135]:
test_f1

0.2237885462555066

In [125]:
lr_test2 = LogisticRegression()
lr_test2.fit(X_train, y_train)
pred_test2 = lr_test2.predict_proba(X_test)[:,1]

c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [144]:
test2_f1 = f1_score(y_true=y_test, y_pred=pred_test2 > 0.09)
test2_f1

0.21339950372208435

In [44]:
param_grid = dict(threshold=[.05,.0023, 0.01, 0.07, 0.08], selector = [LogisticRegression(),KNeighborsClassifier(n_neighbors=2),ExtraTreesClassifier()] )

resultsProbaSelectModel = []
for params in tqdm(ParameterGrid(param_grid)):

    clf = Pipeline([
        ('feature_selection', SelectFromModel(LinearSVC())),
        ('classification', RandomForestClassifier())
    ])
    clf.fit(X_train, y_train)

    resultsProbaSelectModel.append(dict(
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=clf.predict_proba(X_train)[:, 1] > params['threshold']),
                        test_f1 = f1_score(y_true=y_test, y_pred=clf.predict_proba(X_test)[:,1] > params['threshold']),

    ))

  0%|          | 0/15 [00:00<?, ?it/s]c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
  7%|▋         | 1/15 [01:02<14:33, 62.38s/it]c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
 13%|█▎        | 2/15 [02:01<13:09, 60.70s/it]c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
 20%|██        | 3/15 [03:02<12:05, 60.44s/it]c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
 27%|██▋       | 4/15 [04:00<10:55, 59.64s/it]c:\users\w

In [45]:
pd.DataFrame(resultsProbaSelectModel).sort_values('test_f1').style.bar(vmin=0, vmax=1)

,parameters,train_f1,test_f1
1,"{'selector': LogisticRegression(), 'threshold': 0.0023}",0.168175,0.097253
6,"{'selector': KNeighborsClassifier(n_neighbors=2), 'threshold': 0.0023}",0.164791,0.098731
11,"{'selector': ExtraTreesClassifier(), 'threshold': 0.0023}",0.169545,0.101506
12,"{'selector': ExtraTreesClassifier(), 'threshold': 0.01}",0.289933,0.106065
2,"{'selector': LogisticRegression(), 'threshold': 0.01}",0.292485,0.109175
7,"{'selector': KNeighborsClassifier(n_neighbors=2), 'threshold': 0.01}",0.292733,0.109321
13,"{'selector': ExtraTreesClassifier(), 'threshold': 0.07}",0.833575,0.122105
4,"{'selector': LogisticRegression(), 'threshold': 0.08}",0.857143,0.122402
5,"{'selector': KNeighborsClassifier(n_neighbors=2), 'threshold': 0.05}",0.712284,0.123743
9,"{'selector': KNeighborsClassifier(n_neighbors=2), 'threshold': 0.08}",0.855022,0.124294


In [43]:
pd.DataFrame(resultsProba).sort_values('test_f1').style.bar(vmin=0, vmax=1)

,parameters,train_f1,test_f1
12,"{'selector': ExtraTreesClassifier(), 'threshold': 0.01}",0.085220,0.081572
2,"{'selector': LogisticRegression(), 'threshold': 0.01}",0.085178,0.081856
6,"{'selector': KNeighborsClassifier(n_neighbors=2), 'threshold': 0.0023}",0.084972,0.081909
7,"{'selector': KNeighborsClassifier(n_neighbors=2), 'threshold': 0.01}",0.085174,0.082127
1,"{'selector': LogisticRegression(), 'threshold': 0.0023}",0.084918,0.082228
11,"{'selector': ExtraTreesClassifier(), 'threshold': 0.0023}",0.084906,0.082438
13,"{'selector': ExtraTreesClassifier(), 'threshold': 0.07}",0.190345,0.103734
8,"{'selector': KNeighborsClassifier(n_neighbors=2), 'threshold': 0.07}",0.189300,0.107216
10,"{'selector': ExtraTreesClassifier(), 'threshold': 0.05}",0.187883,0.109980
14,"{'selector': ExtraTreesClassifier(), 'threshold': 0.08}",0.191136,0.111801


In [ ]:
pred = clf.predict_proba(X_test)[:,1]

In [ ]:
threshold = 0.07
f1_score(y_true=y_test, y_pred= pred > threshold)

In [ ]:
# https://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes
from sklearn.linear_model import LinearRegression, LogisticRegression, BayesianRidge, PassiveAggressiveRegressor, SGDClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, CategoricalNB, MultinomialNB, ComplementNB

In [ ]:
from sklearn.preprocessing import StandardScaler

std_scaler = MinMaxScaler()

X_train_scaled = std_scaler.fit_transform(X_train)
X_test_scaled = std_scaler.transform(X_test)

X_train_scaled

# todo надо бужет скэйлить!
from sklearn.ensemble import RandomForestRegressor
clf = SGDClassifier()

clf.fit(X_train_scaled,y_train)

In [ ]:
SVC_model = SVC()
# В KNN-модели нужно указать параметр n_neighbors
# Это число точек, на которое будет смотреть
# классификатор, чтобы определить, к какому классу принадлежит новая точка
KNN_model = KNeighborsClassifier(n_neighbors=5)
RFC_model = RandomForestClassifier()

SVC_model.fit(X_train, y_train)
KNN_model.fit(X_train, y_train)
RFC_model.fit(X_train, y_train)

SVC_prediction = SVC_model.predict(X_test)
KNN_prediction = KNN_model.predict(X_test)
RFC_model = RandomForestClassifier.predict_proba(X_test)[:,1]

In [ ]:
logReg = KNeighborsClassifier(n_neighbors=2)
logReg.fit(X_train_scaled, y_train)

In [ ]:
predictions = logReg.predict(X_test_scaled)

In [ ]:
proba = logReg.predict_proba(X_test_scaled)[:, 1]

In [ ]:
threshold = 0.03
f1_score(y_true=y_test, y_pred=proba > threshold)

In [ ]:
f1_score(y_true=y_test, y_pred=KNN_prediction)

In [ ]:
param_grid = dict(threshold=[.1, .2, .3, .4, .5, .01,.005, .05, .001,.0023, 0.003])

resultsProba = []
for params in tqdm(ParameterGrid(param_grid)):

  #  prediction = logReg.predict_proba(X_test_scaled)[:,1]
    resultsProba.append(dict(
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=logReg.predict_proba(X_train_scaled)[:, 1] > params['threshold']),
                        test_f1 = f1_score(y_true=y_test, y_pred=logReg.predict_proba(X_test_scaled)[:,1] > params['threshold']),

    ))


In [ ]:
param_grid = dict(threshold=[.1, .2, .3, .4, .5, .01,.005, .05, .001,.0023, 0.003])

resultsProba = []
for params in tqdm(ParameterGrid(param_grid)):

  #  prediction = logReg.predict_proba(X_test_scaled)[:,1]
    resultsProba.append(dict(
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=logReg.predict_proba(X_train_scaled)[:, 1] > params['threshold']),
                        test_f1 = f1_score(y_true=y_test, y_pred=logReg.predict_proba(X_test_scaled)[:,1] > params['threshold']),

    ))


In [ ]:
pd.DataFrame(resultsProba).sort_values('test_f1').style.bar(vmin=0, vmax=1)



prediction = clf.predict_proba(X_test)[:,1]
#print (f'Test f1 = {f1_score(y_true=y_test, y_pred=nb_pipeline.predict(X_test))}')

prediction

score = clf.score(X_test, y_test)
score

In [ ]:
X_test

In [ ]:

minMaxTest_Y = prep_train_data.is_high_priority

In [ ]:
test_minMaxdf_X = prep_train_data.summary.astype(str) + " " + prep_train_data.description.astype(str)

In [ ]:
X_train_minmax, X_test_minmax, y_train_minmax, y_test_minmax = train_test_split(test_minMaxdf_X, minMaxTest_Y,
                                                    stratify=Y,
                                                    random_state=3,
                                                    test_size=0.25,
                                                    shuffle=True)

In [ ]:
X_train_minmax

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.linear_model import LinearRegression, LogisticRegression, BayesianRidge, PassiveAggressiveRegressor, SGDClassifier
param_grid = dict(min_df=[0.005, 0.01,.05, 0.06], max_df=[.5,.6,.7,.8], classifier=[ComplementNB(),LogisticRegression()], threshold=[.15,.05, 0.1])

results = []

for params in tqdm(ParameterGrid(param_grid)):
#     print(params)
    pipe = Pipeline(steps = [ ('tfid', TfidfVectorizer(min_df= params['min_df'], max_df= params['max_df'],
                                    tokenizer=LemmaTokenizer(),
                             )),
                             ('to_dense', DenseTransformer()),
                             ('clf',  params['classifier'])
                            ]
                       )

    #cv_train_preds = cross_val_predict(pipe, X_text_train, y_train, cv = 5, n_jobs = -1)

    pipe.fit(X_train_minmax, y_train_minmax)


    probabilitiesText1 = pipe.predict_proba(X_train_minmax)[:, 1]
    probabilitiesText2 = pipe.predict_proba(X_test_minmax)[:, 1]

    results.append(dict(
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=probabilitiesText1 > params['threshold']),
                        test_f1 = f1_score(y_true=y_test, y_pred=probabilitiesText2 > params['threshold'])
    ))


In [ ]:
pd.DataFrame(results).sort_values('test_f1').style.bar(vmin=0, vmax=1)

In [ ]:
pd.DataFrame(results).drop(columns='estimator').sort_values('test_f1').style.bar(vmin=0, vmax=1)

In [ ]:


#param_grid = dict(classifier=[ComplementNB(),LogisticRegression(),LinearRegression(), RandomForestRegressor()])




nb_pipeline = Pipeline(steps = [('scaler', MinMaxScaler()),
                                ('clf', ComplementNB())
                               ]
                       )

nb_pipeline.fit(X_train, y_train)

print (f'Train f1 = {f1_score(y_true=y_train, y_pred=nb_pipeline.predict(X_train))}')
print (f'Test f1 = {f1_score(y_true=y_test, y_pred=nb_pipeline.predict(X_test))}')

In [ ]:
param_grid = dict(threshold=[.1, .2, .3, .4, .5, .01,.005, .05])

results = []
for params in tqdm(ParameterGrid(param_grid)):
    probabilities1 = nb_pipeline.predict_proba(X_train)[:, 1]
    probabilities2 = nb_pipeline.predict_proba(X_test)[:, 1]
    results.append(dict(
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=probabilities1 > params['threshold']),
                        test_f1 = f1_score(y_true=y_test, y_pred=probabilities2 > params['threshold'])
    ))



In [ ]:
pd.DataFrame(results).sort_values('test_f1').style.bar(vmin=0, vmax=1)

In [ ]:


from sklearn.linear_model import LinearRegression, LogisticRegression, BayesianRidge, PassiveAggressiveRegressor, SGDClassifier
param_grid = dict(min_df=[0.01, .05], max_df=[.7, .8], scaler=[MinMaxScaler()])

results = []

for params in tqdm(ParameterGrid(param_grid)):
#     print(params)
    pipe = Pipeline(steps = [
        ('scaler', params['scaler']),
                             ('clf', ComplementNB())
                            ]
                       )

    #cv_train_preds = cross_val_predict(pipe, X_text_train, y_train, cv = 5, n_jobs = -1)

    pipe.fit(X_train, y_train)
    train_preds = pipe.predict(X_test)

    results.append(dict(
                        estimator=pipe,
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=pipe.predict(X_train)),
                        test_f1 = f1_score(y_true=y_test, y_pred=pipe.predict(X_test))
    ))


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.linear_model import LinearRegression, LogisticRegression, BayesianRidge, PassiveAggressiveRegressor, SGDClassifier
param_grid = dict(min_df=[0.01, .05], max_df=[.7, .8])

results = []

for params in tqdm(ParameterGrid(param_grid)):
#     print(params)
    pipe = Pipeline(steps = [('tfidf', TfidfVectorizer(min_df=params['min_df'], max_df=params['max_df'],
                                                       token_pattern=r'[A-Za-z]{2,}',
                                                       stop_words=STOP_WORDS)),
                             ('to_dense', DenseTransformer()),
                             ('clf', ComplementNB())
                            ]
                       )

    #cv_train_preds = cross_val_predict(pipe, X_text_train, y_train, cv = 5, n_jobs = -1)

    pipe.fit(X_train, y_train)
    train_preds = pipe.predict(X_train)

    results.append(dict(
                        estimator=pipe,
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=pipe.predict(X_train)),
                        test_f1 = f1_score(y_true=y_test, y_pred=pipe.predict(X_test))
    ))

In [ ]:
pd.DataFrame(results).drop(columns='estimator').sort_values('test_f1').style.bar(vmin=0, vmax=1)

In [ ]:
train_data1

In [ ]:
probabilities = pipe.predict_proba(X)[:, 1]

In [ ]:
probabilities

In [ ]:
threshold = 0.6

In [ ]:
f1_score(y_true=y, y_pred=probabilities > threshold)